In [1]:
%%capture
%run geo_coil.ipynb

In [10]:
import sys
sys.path.insert(0,'../../') # adds parent directory
import pde
MESH = pde.mesh.netgen(geoOCCmesh)
print(MESH)

np:657, nt:1222, nq:0, ne:212, ne_all:1878
